In [2]:
### Plotting bettis

script_start_time = time()
println("\nimporting packages...")

using Pkg
using Statistics
using LinearAlgebra
using Eirene
using StatsBase
using JLD
using Plots
Pkg.add("ColorSchemes")
using ColorSchemes
using JSON

include("helper_functions.jl")
println("packages and functions imported")
printstyled("Elapsed time = $(time() - script_start_time) seconds \n \n", color = :yellow)


importing packages...
  Updating registry at `/opt/julia/registries/General`
 Resolving package versions...


┌ Warning: Some registries failed to update:
│     — `/opt/julia/registries/General` — registry dirty
└ @ Pkg.Types /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.2/Pkg/src/Types.jl:1171


  Updating `/opt/julia/environments/v1.2/Project.toml`
  [35d6a980] + ColorSchemes v3.9.0
  Updating `/opt/julia/environments/v1.2/Manifest.toml`
 [no changes]
packages and functions imported
Elapsed time = 9.754185914993286 seconds 
 


In [5]:
### Set parameters

# Read from config file
config_file = "config090920.json"
config = read_config("$(homedir())/configs/$(config_file)")

# Parameters for all graphs
const NNODES = config["NNODES"]
const MAXDIM = config["MAXDIM"]
const NREPS = config["NREPS"]
const DATE_STRING = config["DATE_STRING"]


### Locate data and find nametags
read_dir = "../processed_data/results/$(NNODES)nodes"
betti_files = filter(x->occursin("_bettis.jld",x), readdir(read_dir))
betti_files = filter(x->occursin(DATE_STRING, x), betti_files)
println("Located the following graph files:")
for betti_file in betti_files
    println(betti_file)
end


# Locate the nametags
nametags = []
for betti_file in betti_files
    println(betti_file)
    tag = split(split(betti_file, "$(DATE_STRING)_")[2], "_bettis")[1]
    nametags = [nametags; tag]
end

nametags = unique(nametags)


Located the following graph files:
IID_70nodes_20reps_090920_backward_bettis.jld
IID_70nodes_20reps_090920_forward_bettis.jld
IID_70nodes_20reps_090920_thresh03_edge725_noiseOnly_bettis.jld
IID_70nodes_20reps_090920_thresh03_edge725_threshold_bettis.jld
IID_70nodes_20reps_090920_thresh04_edge966_noiseOnly_bettis.jld
IID_70nodes_20reps_090920_thresh04_edge966_threshold_bettis.jld
IID_70nodes_20reps_090920_thresh05_edge1208_noiseOnly_bettis.jld
IID_70nodes_20reps_090920_thresh05_edge1208_threshold_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_backward_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_forward_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_thresh03_edge725_noiseOnly_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_thresh03_edge725_threshold_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_thresh04_edge966_noiseOnly_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_thresh04_edge966_threshold_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_thresh05_edge1208

8-element Array{Any,1}:
 "backward"                   
 "forward"                    
 "thresh03_edge725_noiseOnly" 
 "thresh03_edge725_threshold" 
 "thresh04_edge966_noiseOnly" 
 "thresh04_edge966_threshold" 
 "thresh05_edge1208_noiseOnly"
 "thresh05_edge1208_threshold"

In [6]:


for nametag in nametags
    
    ## Create one large plot with all of the betti curves
    betti_files_nametag = filter(x -> occursin("$(nametag)",x), betti_files)
    model_names = [split(betti_file_nametag,"_")[1] for betti_file_nametag in betti_files_nametag]
    
    nModels = length(betti_files_nametag)
    nEdges = binomial(NNODES, 2)
    bettisAll = zeros(NREPS, nEdges, MAXDIM, nModels)
    
    for (i,betti_file_nametag) in enumerate(betti_files_nametag)

        # Read in Betti curves
        betti_dict = load("$(read_dir)/$(betti_file_nametag)")
        bettisAll[:,:,:, i] = betti_dict["bettisArray"]
        println(betti_file_nametag)

    end

    bettisAllAvg = dropdims(mean(bettisAll, dims=1), dims=1)
    println("Size bettisAllAvg is $(size(bettisAllAvg))")

    # Finally plot
    p1a = plot(bettisAllAvg[:,1,:], layout = 12, size = (1000,800), title = ["$(model_names[i])" for j=1:1, i=1:nModels])
    for d=2:MAXDIM
        plot!(bettisAllAvg[:,d,:],legend = nothing, xlabel = "Edges added", ylabel="Beta_k")

    end
    
    # If the file is of threshold type, add the threshold line
    if occursin("threshold", nametag)
        
        # Get threshold edge
        thresh_string = split(split(nametag,"edge")[2],"_")[1]
        threshold_edge = parse(Int, thresh_string)
        println("processing threshold edge $(threshold_edge)")
        
        # Plot the vertical line
        vline!(transpose([threshold_edge for i=1:nModels]), color=:gray)
    end


    savefig("../figures/all_bettis_$(nametag).pdf")
end






IID_70nodes_20reps_090920_backward_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_backward_bettis.jld
discreteUniformConf_70nodes_20reps_0_1000_090920_backward_bettis.jld
dotProduct_70nodes_20reps_3_090920_backward_bettis.jld
geometricConf_70nodes_20reps_001_100_090920_backward_bettis.jld
randomGeom_70nodes_20reps_3_090920_backward_bettis.jld
Size bettisAllAvg is (2415, 3, 6)
IID_70nodes_20reps_090920_forward_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_forward_bettis.jld
discreteUniformConf_70nodes_20reps_0_1000_090920_forward_bettis.jld
dotProduct_70nodes_20reps_3_090920_forward_bettis.jld
geometricConf_70nodes_20reps_001_100_090920_forward_bettis.jld
randomGeom_70nodes_20reps_3_090920_forward_bettis.jld
ringLattice_70nodes_20reps_090920_forward_bettis.jld
Size bettisAllAvg is (2415, 3, 7)
IID_70nodes_20reps_090920_thresh03_edge725_noiseOnly_bettis.jld
cosineGeometric_70nodes_20reps_3_090920_thresh03_edge725_noiseOnly_bettis.jld
discreteUniformConf_70nodes_20reps_0_1000_090

In [5]:
a = []
append!(a,["two"])

1-element Array{Any,1}:
 "two"